In [1]:

temp = 'C:/Users/vjosv/master/top_100_images_with_location_data.csv'
import time
import torch 
import torchvision
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import os
import torch
import pandas as pd
import numpy as np
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.backends.cudnn as cudnn
import torch.nn as nn

import torch.nn.functional as F
import torch.nn.parallel
import torch.optim as optim
import torchvision
import torchvision.models as models
import sys
import copy

import ast
import shutil
from PIL import Image
from averagemeter import *
from models import *
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.autograd import Variable
from torch.utils.data import sampler
from torchvision import datasets
from torchvision import transforms
import torchvision
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


INPUT_SIZE = 224
NUM_CLASSES = 150
NUM_EPOCHS = 30
LEARNING_RATE = 1e-1
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
best_prec1 = 0
classes = []

train_dat = pd.read_csv('C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_train.csv')
test_dat = pd.read_csv('C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_test.csv')
df_comb = pd.concat([train_dat,test_dat])


True


In [11]:
transforms = [T.ToTensor(),T.Resize((INPUT_SIZE,INPUT_SIZE))]

composed = T.Compose(transforms)




landscape_val = df_comb['landscape'].unique()


one_hot_landscape_dict = {}
for i,lv in enumerate(landscape_val):
    zero = np.zeros(len(landscape_val))
    zero[i]=1 
    one_hot_landscape_dict[str(lv)]= zero

artreslag_val =df_comb['artreslag'].unique()
one_hot_artreslag_dict = {}
for i,av in enumerate(artreslag_val):
    zero = np.zeros(len(artreslag_val))
    zero[i]=1 
    one_hot_artreslag_dict[str(av)]= zero

arjordbr_val =df_comb['arjordbr'].unique()
one_hot_arjordbr_dict = {}
for i,av in enumerate(arjordbr_val):
    zero = np.zeros(len(arjordbr_val))
    zero[i]=1 
    one_hot_arjordbr_dict[str(av)]= zero
    
    

arskogbon_val =df_comb['arskogbon'].unique()
one_hot_arskogbon_dict = {}
for i,av in enumerate(arskogbon_val):
    zero = np.zeros(len(arskogbon_val))
    zero[i]=1 
    one_hot_arskogbon_dict[str(av)]= zero
    
treeline_val =df_comb['treeline'].unique()
one_hot_treeline_dict = {}
for i,av in enumerate(treeline_val):
    zero = np.zeros(len(treeline_val))
    zero[i]=1 
    one_hot_treeline_dict[str(av)]= zero
    

square_10x10_val =df_comb['square_10x10'].unique()
one_hot_square_10x10_dict = {}
for i,av in enumerate(square_10x10_val):
    zero = np.zeros(len(square_10x10_val))
    zero[i]=1 
    one_hot_square_10x10_dict[str(av)]= zero


arveget_val =df_comb['arveget'].unique()
one_hot_arveget_dict = {}
for i,av in enumerate(arveget_val):
    zero = np.zeros(len(arveget_val))
    zero[i]=1 
    one_hot_arveget_dict[str(av)]= zero

artype_val =df_comb['artype'].unique()
one_hot_artype_dict = {}
for i,av in enumerate(artype_val):
    zero = np.zeros(len(artype_val))
    zero[i]=1 
    one_hot_artype_dict[str(av)]= zero


class ImagesWithLocationDataset(Dataset):
    """Plant images with location data."""
    
    
    def __init__(self, csv_file, root_dir, transform=None):
    

        self.location_dataframe = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.location_dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.location_dataframe['path'].iloc[idx]
        
        image = io.imread(img_name)
        location_data = np.array(ast.literal_eval(self.location_dataframe['in_1000'].iloc[idx]))
        landscape = str(self.location_dataframe['landscape'].iloc[idx])
        artreslag = str(self.location_dataframe['artreslag'].iloc[idx])
        arjordbr = str(self.location_dataframe['arjordbr'].iloc[idx])
        target = np.array(self.location_dataframe['target'].iloc[idx])
        height = self.location_dataframe['height'].iloc[idx]
        north = self.location_dataframe['Nordkoordinat'].iloc[idx]
        artype = str(self.location_dataframe['artype'].iloc[idx])
        treeline = str(self.location_dataframe['treeline'].iloc[idx])
        square_10x10 =self.location_dataframe['square_10x10'].iloc[idx]
        arveget = str(self.location_dataframe['arveget'].iloc[idx])
        arskogbon =  str(self.location_dataframe['arskogbon'].iloc[idx])
        # landmarks = self.landmarks_frame.iloc[idx, 1:]
        # landmarks = np.array([landmarks], dtype=float).reshape(-1, 2)
        sample = {'image': image, 'location_data': location_data, 'target':target, 'landscape':landscape,
                 'artreslag':artreslag, 'arjordbr':arjordbr, 'height':height,'north':north,
                 'artype':artype,'treeline':treeline, 'square_10x10':square_10x10, 'arveget':arveget,
                 'arskogbon':arskogbon}
        # sample = {'image': image, 'target':target}
        if self.transform:
            sample['image'] = self.transform(sample['image'])
            sample['target'] = torch.from_numpy(sample['target'])
            sample['location_data'] = torch.from_numpy(location_data).float()
            sample['landscape'] = torch.from_numpy(one_hot_landscape_dict[landscape]).float()
            sample['artreslag'] = torch.from_numpy(one_hot_artreslag_dict[artreslag]).float()
            sample['arjordbr'] = torch.from_numpy(one_hot_arjordbr_dict[arjordbr]).float()
            sample['artype'] = torch.from_numpy(one_hot_artype_dict[artype]).float()
            sample['treeline'] = torch.from_numpy(one_hot_treeline_dict[treeline]).float()
            sample['square_10x10'] = torch.from_numpy(one_hot_square_10x10_dict[square_10x10]).float()
            sample['arveget'] = torch.from_numpy(one_hot_arveget_dict[arveget]).float()
            sample['arskogbon'] = torch.from_numpy(one_hot_arskogbon_dict[arskogbon]).float()
            sample['height'] = torch.from_numpy(np.array([height])).float()
            sample['north'] = torch.from_numpy(np.array([north])).float()
        return sample
    
    def from_np_array(self,array_string):
        array_string = ','.join(array_string.replace('[ ', '[').split())
        return np.array(ast.literal_eval(array_string))
    
    def name(self):
        return 'ImagesWithLocationDataset'

class ImagesWithLocationDataset_conv(Dataset):
    """Plant images with location data."""
    
    
    def __init__(self, csv_file, root_dir, transform=None):
    

        self.location_dataframe = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.location_dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.location_dataframe['path'].iloc[idx]
        
        image = io.imread(img_name)
        location_data = self.from_np_array(self.location_dataframe['count_in_1000'].iloc[idx])
        
        map_square = self.split_map_square(location_dataframe['map_square'].iloc[idx])
        
 
        target = np.array(self.location_dataframe['target'].iloc[idx])
        
        

        # landmarks = self.landmarks_frame.iloc[idx, 1:]
        # landmarks = np.array([landmarks], dtype=float).reshape(-1, 2)
        sample = {'image': image, 'location_data': location_data, 'map_square': map_square,'target':target}

        if self.transform:
            sample['image'] = self.transform(sample['image'])
            sample['target'] = torch.from_numpy(sample['target'])
            sample['location_data'] = torch.from_numpy(location_data).float()
            
            for t in self.transform.transforms:
                if isinstance(t,torchvision.transforms.transforms.Resize):
                    input_size = t.size[0]
            map_s = np.zeros((input_size,input_size))
            map_s[:input_size//2]=map_square[0]
            map_s[input_size//2:]=map_square[1]
            map_s = torch.from_numpy(map_s).float()
            map_s = map_s.expand(1,input_size,input_size)
            sample['map_square'] = map_s
        return sample
    
    def from_np_array(self,array_string):
        array_string = ','.join(array_string.replace('[ ', '[').split())
        return np.array(ast.literal_eval(array_string))
    
    def split_map_square(self,list_string):
        v = list_string.split(',')
        return [int(v[0][1:]),int(v[1][:-1])]

    def name(self):
        return 'ImagesWithLocationDataset_conv'


In [12]:
# This file provides a long_running decorator to indicate that a function needs a long amount of time to complete and
# the computer should not enter standby. This file currently only works on Windows and is a no-op on other platforms.

#copied from Kevin Barnes/kbarnes3: https://gist.github.com/kbarnes3/3fb7d353e9bdd3efccd5

import ctypes
import platform

ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001


def _set_thread_execution(state):
    ctypes.windll.kernel32.SetThreadExecutionState(state)


def prevent_standby():
    if platform.system() == 'Windows':
        _set_thread_execution(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)


def allow_standby():
    if platform.system() == 'Windows':
        _set_thread_execution(ES_CONTINUOUS)


def long_running(func):
    def inner(*args, **kwargs):
        prevent_standby()
        result = func(*args, **kwargs)
        allow_standby()
        return result
    return inner

In [13]:
image_df = pd.read_csv('C:/Users/vjosv/master/top_100_images_with_location_data.csv')
path_list = list(image_df['path'])

non_existing_files = []
for i,pl in enumerate(path_list):
    if not os.path.isfile(pl):
        non_existing_files.append((i,pl))
        
non_existing_files


[]

In [14]:

train_dat = pd.read_csv('C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_train.csv')
test_dat = pd.read_csv('C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_test.csv')
df_comb = pd.concat([train_dat,test_dat])

In [6]:
# dat = pd.read_csv('C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_train.csv')


image_dataset = ImagesWithLocationDataset('C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_train.csv','s',transform=composed)




In [7]:
train_loader = torch.utils.data.DataLoader(image_dataset, batch_size=16, shuffle=True, num_workers=0)

In [8]:
for i,input in enumerate(train_loader):
    
    # print(input)
    l = ['image','target']
    a = torch.cat([input[k] for k in input if k not in l],1 )
    a
    break
    
    


In [9]:
a.shape

torch.Size([16, 170])

In [10]:
def train(train_loader, model, criterion, optimizer, epoch):

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, input in enumerate(train_loader):
        # measure data loading time
        # print(input['image'].shape)
        if USE_CUDA:
            image = input['image'].cuda(non_blocking=True)
            target = input['target'].cuda(non_blocking=True)
            
            l = ['image','target']
            location_data = torch.cat([input[k].cuda(non_blocking=True) for k in input if k not in l],1 )
            
            # l = ['height','north']
            # location_data = torch.cat([input[k] for k in input if k not in l],1 )

            # location_data = torch.cat([input[k].cuda(non_blocking=True) for k in input if k in l],1 )
            # location_data = input['location_data'].cuda(non_blocking=True)
            # map_square = input['map_square'].cuda(non_blocking=True)
        else:
            image = input['image']
            target = input['target']
            l = ['image','target']
            l = ['height','north']
            # location_data = torch.cat([input[k] for k in input if k not in l],1 )

            location_data = torch.cat([input[k] for k in input if k in l],1 )
            # location_data = input['location_data']
            # map_square = input['map_square']
        
        

        input_var = torch.autograd.Variable(image)
        target_var = torch.autograd.Variable(target)
        location_data_var = torch.autograd.Variable(location_data)
        # map_square_var = torch.autograd.Variable(map_square)
        # location_data_var = torch.autograd.Variable(location_data)
        # compute output
        # output = model(input_var,map_square)

        output = model(input_var,location_data_var)
        data_time.update(time.time() - end)

        
        # compute output
        # print(input_var.shape)
        # print(output)

        loss = criterion(output, target_var)
        # print(loss)
        # if i >10:
        #     break
        
        save_output_target_train(model,output,target_var,epoch,i)
        # measure accuracy and record loss
        prec1, prec5,conf = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data.item(), image.size(0))
        top1.update(prec1.item(), image.size(0))
        top5.update(prec5.item(), image.size(0))
        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  '\Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1, top5=top5))
            
            
def validate(val_loader, model, criterion,epoch,save_output=False):
    # global output_aug_kde001, sample_pos, within_square, weight_tensor_kde001,zero_tensor_kde001,tt
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()


    conf = AverageMeter()
    class_correct = list(0. for i in range(200))
    class_total = list(0. for i in range(200))

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, input in enumerate(val_loader):
        

        if USE_CUDA:
            image = input['image'].cuda(non_blocking=True)
            target = input['target'].cuda(non_blocking=True)
            l = ['image','target']
            location_data = torch.cat([input[k].cuda(non_blocking=True) for k in input if k not in l],1 )
            location_data = input['location_data'].cuda(non_blocking=True)
            # map_square = input['map_square'].cuda(non_blocking=True)
            # l = ['height','north']
            # location_data = torch.cat([input[k] for k in input if k not in l],1 )

            # location_data = torch.cat([input[k].cuda(non_blocking=True) for k in input if k in l],1 )
        else:
            image = input['image']
            target = input['target']
            l = ['image','target']
            location_data = torch.cat([input[k] for k in input if k not in l],1 )
            
            # l = ['height','north']
            # location_data = torch.cat([input[k] for k in input if k in l],1 )
            # location_data = input['location_data']
            # map_square = input['map_square']
        input_var = torch.autograd.Variable(image)
        target_var = torch.autograd.Variable(target)
        location_data_var = torch.autograd.Variable(location_data)
        # location_data_var = torch.autograd.Variable(location_data)
        # map_square_var = torch.autograd.Variable(map_square)
        # compute output
        # output = model(input_var,map_square)
        output = model(input_var,location_data_var)
        loss = criterion(output, target)
        
        save_output_target_validate(model,output,target,epoch,i)
        

        # measure accuracy and record loss
        
        prec1, prec5, conf1 = accuracy(output.data, target, topk=(1, 5))
        

        losses.update(loss.data.item(), image.size(0))
        
        top1.update(prec1.item(), image.size(0))
        top5.update(prec5.item(), image.size(0))


        conf.update(conf1,1)
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:

            print('Test: [{0}/{1}]\n'.format(i, len(val_loader)))
            print('Time {batch_time.val:.3f} ({batch_time.avg:.3f})\n'.format(batch_time=batch_time))
            print('Loss {loss.val:.4f} ({loss.avg:.4f})\n'.format(loss=losses))
            print('Prec@1 {top1.val:.3f} ({top1.avg:.3f})\n'.format(top1=top1))
            print('Prec@5 {top5.val:.3f} ({top5.avg:.3f})\n'.format(top5=top5))

        # print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
        #       .format(top1=top1, top5=top5))
        


    return top1.avg

In [15]:
def adjust_learning_rate(optimizer, epoch, adjust_now, LEARNING_RATE=LEARNING_RATE):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LEARNING_RATE
    if (epoch-1) < 0:
        former_lr=optimizer.param_groups[0]['lr']

    former_lr = copy.copy(optimizer.param_groups[0]['lr'])
    if adjust_now:
        former_lr = copy.copy(optimizer.param_groups[0]['lr'])
        LEARNING_RATE = LEARNING_RATE*0.1
        lr = LEARNING_RATE
    print('\n[Learning Rate] {:0.6f}'.format(lr))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr != former_lr, LEARNING_RATE


def save_checkpoint(state, is_best, model):
    filename=f'saved_models/{model.name}_checkpoint.pth.tar'
    torch.save(state, filename)
    if is_best:
        print('\n[INFO] Saved Model to model_best.pth.tar')
        shutil.copyfile(filename, f'saved_models/{model.name}_model_best.pth.tar')
        
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred_t = pred.t()

    correct = pred_t.eq(target.view(1, -1).expand_as(pred_t))

    nb_classes = output.shape[1]

    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    _, preds = torch.max(output, 1)
    
    for t, p in zip(target.view(-1), preds.view(-1)):
        confusion_matrix[t.long(), p.long()] += 1

    # print(confusion_matrix)
    
    
    # print(confusion_matrix.diag()/confusion_matrix.sum(1))
        
    res = []
    for k in topk:
        correct_k = torch.sum(torch.reshape(correct[:k],(-1,) ).float()) #.double().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    res.append(confusion_matrix)
    return res

In [16]:

def save_checkpoint(state, is_best, model):
    filename=f'saved_models/{model.name}_checkpoint.pth.tar'
    torch.save(state, filename)
    if is_best:
        print('\n[INFO] Saved Model to model_best.pth.tar')
        shutil.copyfile(filename, f'saved_models/{model.name}_model_best.pth.tar')



In [17]:
def save_output_target_validate(model,output,target,epoch,batch):
        torch.save(output,f'saved_output/{model.name}_validating_output/output_e{epoch}_b{batch}')
        torch.save(target,f'saved_output/{model.name}_validating_output/target_b{batch}')
        
def save_output_target_train(model,output,target,epoch,batch):
        torch.save(output,f'saved_output/{model.name}_training_output/output_e{epoch}_b{batch}')

        torch.save(target,f'saved_output/{model.name}_training_output/target_e_{epoch}_b{batch}')
      

In [18]:

'combined_location_image_df_top100.csv'


torch.cuda.empty_cache() 
seed =1711
split_train =80
split_val =100-split_train
numb_targets = 100


model = models.resnet101(weights="IMAGENET1K_V1")


optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()




model.fc = nn.Linear(in_features=2048, out_features=100, bias=True)

model.name = f'resnet101_v4_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2'


if USE_CUDA:
    model.cuda()


test_path ='C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_test.csv'

train_path =  'C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_train.csv'
# test_dat_path = 'C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_test.csv'

train_image_dataset = ImagesWithLocationDataset(train_path,'s',transform=composed)
test_image_dataset = ImagesWithLocationDataset(test_path,'s',transform=composed)

torch.manual_seed(seed)
# train_size = int(split_train*0.01 * len(image_dataset))
# test_size = len(image_dataset) - train_size
# data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=8, shuffle=False, num_workers=0)


USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()

    
@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS):
 
    train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=12, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=12, shuffle=False, num_workers=0)
 
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    
    print('\n[INFO] Training Started')
    five_epochs_since_best = False
    epochs_since_best = 0
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
        
        adjusted_rate,LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best,LEARNING_RATE)
        
        if adjusted_rate:
            print('Adjusted Learning rate resume from best model')
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        # torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1
            

if USE_CUDA:
    model.cuda()

# args_resume = f'C:/Users/vjosv/master/Deep-Leafsnap/saved_models/{model.name}_model_best.pth.tar'
# print("=> loading checkpoint '{}'".format(args_resume))
# checkpoint = torch.load(args_resume)
# best_prec1 = checkpoint['best_prec1']
# model.load_state_dict(checkpoint['state_dict'])
# print("=> loaded checkpoint '{}' (epoch {})"
#       .format(args_resume, checkpoint['epoch']))
# epoch = checkpoint['epoch']



model.name = f'resnet101_with_full_loc_dat'


numb_targets = 100
location_vals = 170
location_fc = nn.Sequential(
          nn.Linear(numb_targets+location_vals, (numb_targets+location_vals)),
          nn.ReLU(inplace=True),
          # nn.Dropout(p=0.5, inplace=False),
          nn.Linear((numb_targets+location_vals),(numb_targets+location_vals)),
          nn.ReLU(inplace=True),
          # nn.Dropout(p=0.5, inplace=False),
            nn.Linear((numb_targets+location_vals),(numb_targets+location_vals)),
              nn.ReLU(inplace=True),
          # nn.Dropout(p=0.5, inplace=False),
            nn.Linear((numb_targets+location_vals),numb_targets),
        )

model.add_module('location_fc',location_fc)




location_fc1 = nn.Sequential(
          nn.Linear(170, 170*2),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.5, inplace=False),
          nn.Linear(170*2,170),
            nn.ReLU(inplace=True),
          nn.Dropout(p=0.5, inplace=False),
          nn.Linear(170,170),
        )

model.add_module('location_fc1',location_fc1)


location_fc2 = nn.Sequential(

          nn.Linear(100,100),
          nn.ReLU(inplace=True),
        nn.Dropout(p=0.5, inplace=False),
            nn.Linear(100,100),
        )

# lin_add = LinearAddLayer(100,100)

# model.add_module('linear_add',lin_add)
# model.add_module('location_fc',location_fc)
# model.add_module('location_fc1',location_fc1)

# model.add_module('location_fc2',location_fc2)

def forward(self, x: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
    # Reshape and permute the input tensor
    x = self._forward_impl(x)
    # x = self.location_fc1(x)
    # x = self.linear_add(x,x5)
    
    x2 = self.location_fc1(x2)
    
    combined = torch.cat((x.view(x.size(0), -1),
                      x2.view(x2.size(0), -1)),
                      dim=1)
    x = self.location_fc(combined)
    return x

import functools


# model.forward = forward
model.forward = functools.partial(forward, model)




USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()
    
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')

#freezing transformer part
for param in model.parameters():
    param.requires_grad = False

for param in model.location_fc.parameters():
    param.requires_grad = True
    

for param in model.location_fc1.parameters():
    param.requires_grad = True  
    
if USE_CUDA:
    model.cuda()
# train_model_square(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 1)

train_model(model,LEARNING_RATE = 1e-7,NUM_EPOCHS = 20)

=> no checkpoint found at 'saved_models/resnet101_with_full_loc_dat_checkpoint.pth.tar'

[INFO] Training Started

[Learning Rate] 0.000000


KeyboardInterrupt: 

In [19]:

'combined_location_image_df_top100.csv'


torch.cuda.empty_cache() 
seed =1711
split_train =80
split_val =100-split_train
numb_targets = 100


model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101')


optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()




model.fc = nn.Linear(in_features=2048, out_features=100, bias=True)

model.name = f'resnet101_v4_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2'


if USE_CUDA:
    model.cuda()


test_path ='C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_test.csv'

train_path =  'C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_train.csv'
# test_dat_path = 'C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_test.csv'

train_image_dataset = ImagesWithLocationDataset(train_path,'s',transform=composed)
test_image_dataset = ImagesWithLocationDataset(test_path,'s',transform=composed)

torch.manual_seed(seed)
# train_size = int(split_train*0.01 * len(image_dataset))
# test_size = len(image_dataset) - train_size
# data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=8, shuffle=False, num_workers=0)


USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()

    
@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS):
 
    train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=12, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=12, shuffle=False, num_workers=0)
 
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    
    print('\n[INFO] Training Started')
    five_epochs_since_best = False
    epochs_since_best = 0
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
        
        adjusted_rate,LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best,LEARNING_RATE)
        
        if adjusted_rate:
            print('Adjusted Learning rate resume from best model')
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        # torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1
            

if USE_CUDA:
    model.cuda()

args_resume = f'C:/Users/vjosv/master/Deep-Leafsnap/saved_models/{model.name}_model_best.pth.tar'
print("=> loading checkpoint '{}'".format(args_resume))
checkpoint = torch.load(args_resume)
best_prec1 = checkpoint['best_prec1']
model.load_state_dict(checkpoint['state_dict'])
print("=> loaded checkpoint '{}' (epoch {})"
      .format(args_resume, checkpoint['epoch']))
epoch = checkpoint['epoch']



model.name = f'resnet101_v4_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2_location4'


numb_targets = 100
location_vals = 170
location_fc = nn.Sequential(
          nn.Linear(numb_targets+location_vals, (numb_targets+location_vals)),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.5, inplace=False),
          nn.Linear((numb_targets+location_vals),(numb_targets+location_vals)),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.5, inplace=False),
            nn.Linear((numb_targets+location_vals),(numb_targets+location_vals)),
              nn.ReLU(inplace=True),
          nn.Dropout(p=0.5, inplace=False),
            nn.Linear((numb_targets+location_vals),numb_targets),
        )

model.add_module('location_fc',location_fc)




location_fc1 = nn.Sequential(
          nn.Linear(170, 170),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.5, inplace=False),
          nn.Linear(170,170),
        )
model.add_module('location_fc1',location_fc1)


location_fc2 = nn.Sequential(

          nn.Linear(100,100),
          nn.ReLU(inplace=True),
        nn.Dropout(p=0.5, inplace=False),
            nn.Linear(100,100),
        )

# lin_add = LinearAddLayer(100,100)

# model.add_module('linear_add',lin_add)
model.add_module('location_fc',location_fc)
# model.add_module('location_fc1',location_fc1)

# model.add_module('location_fc2',location_fc2)

def forward(self, x: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
    # Reshape and permute the input tensor
    x = self._forward_impl(x)
    # x = self.location_fc1(x)
    # x = self.linear_add(x,x5)
    
    x2 = self.location_fc1(x2)
    combined = torch.cat((x.view(x.size(0), -1),
                      x2.view(x2.size(0), -1)),
                      dim=1)
    x = self.location_fc(combined)
    return x

import functools


# model.forward = forward
model.forward = functools.partial(forward, model)




USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()
    
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')

# freezing trained part
for param in model.parameters():
    param.requires_grad = False

for param in model.location_fc.parameters():
    param.requires_grad = True
for param in model.location_fc1.parameters():
    param.requires_grad = True
      
    
if USE_CUDA:
    model.cuda()
# train_model_square(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 1)
train_model(model,LEARNING_RATE = 1e-6,NUM_EPOCHS = 10)
# train_model(model,LEARNING_RATE = 1e-6,NUM_EPOCHS = 20)

Using cache found in C:\Users\vjosv/.cache\torch\hub\pytorch_vision_v0.10.0


=> loading checkpoint 'C:/Users/vjosv/master/Deep-Leafsnap/saved_models/resnet101_v4_100targets_weights_seed1711_split8020_attempt2_model_best.pth.tar'
=> loaded checkpoint 'C:/Users/vjosv/master/Deep-Leafsnap/saved_models/resnet101_v4_100targets_weights_seed1711_split8020_attempt2_model_best.pth.tar' (epoch 17)
=> no checkpoint found at 'saved_models/resnet101_v4_100targets_weights_seed1711_split8020_attempt2_location4_checkpoint.pth.tar'

[INFO] Training Started

[Learning Rate] 0.000001
Epoch: [0][0/1334]	\Time 0.671 (0.671)	Data 0.511 (0.511)	Loss 38443.2383 (38443.2383)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [0][100/1334]	\Time 0.271 (0.289)	Data 0.196 (0.221)	Loss 4.8937 (1294.6233)	Prec@1 0.000 (1.073)	Prec@5 0.000 (4.868)
Epoch: [0][200/1334]	\Time 0.335 (0.290)	Data 0.257 (0.220)	Loss 4.5914 (652.8622)	Prec@1 0.000 (0.912)	Prec@5 8.333 (4.643)
Epoch: [0][300/1334]	\Time 0.319 (0.288)	Data 0.254 (0.218)	Loss 4.6345 (437.5163)	Prec@1 0.000 (0.969)	Prec@5 0.000 (4.734)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (12x100 and 170x170)

In [109]:

'combined_location_image_df_top100.csv'


torch.cuda.empty_cache() 
seed =1711
split_train =80
split_val =100-split_train
numb_targets = 100


model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101')


optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()




model.fc = nn.Linear(in_features=2048, out_features=100, bias=True)

model.name = f'resnet101_v4_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2'


if USE_CUDA:
    model.cuda()


test_path ='C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_test.csv'

train_path =  'C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_train.csv'
# test_dat_path = 'C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_test.csv'

train_image_dataset = ImagesWithLocationDataset(train_path,'s',transform=composed)
test_image_dataset = ImagesWithLocationDataset(test_path,'s',transform=composed)

torch.manual_seed(seed)
# train_size = int(split_train*0.01 * len(image_dataset))
# test_size = len(image_dataset) - train_size
# data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=8, shuffle=False, num_workers=0)


USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()

    
@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS):
 
    train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=12, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=12, shuffle=False, num_workers=0)
 
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    
    print('\n[INFO] Training Started')
    five_epochs_since_best = False
    epochs_since_best = 0
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
        
        adjusted_rate,LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best,LEARNING_RATE)
        
        if adjusted_rate:
            print('Adjusted Learning rate resume from best model')
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        # torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1
            

if USE_CUDA:
    model.cuda()

args_resume = f'C:/Users/vjosv/master/Deep-Leafsnap/saved_models/{model.name}_model_best.pth.tar'
print("=> loading checkpoint '{}'".format(args_resume))
checkpoint = torch.load(args_resume)
best_prec1 = checkpoint['best_prec1']
model.load_state_dict(checkpoint['state_dict'])
print("=> loaded checkpoint '{}' (epoch {})"
      .format(args_resume, checkpoint['epoch']))
epoch = checkpoint['epoch']



model.name = f'resnet101_v4_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2_location3'


numb_targets = 100
location_vals = 170
location_fc = nn.Sequential(
          nn.Linear(numb_targets+location_vals, (numb_targets+location_vals)),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.5, inplace=False),
          nn.Linear((numb_targets+location_vals),(numb_targets+location_vals)),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.5, inplace=False),
            nn.Linear((numb_targets+location_vals),(numb_targets+location_vals)),
              nn.ReLU(inplace=True),
          nn.Dropout(p=0.5, inplace=False),
            nn.Linear((numb_targets+location_vals),numb_targets),
        )

model.add_module('location_fc',location_fc)




location_fc1 = nn.Sequential(
          nn.Linear(170, 170),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.5, inplace=False),
          nn.Linear(170,170),
        )
model.add_module('location_fc1',location_fc1)


location_fc2 = nn.Sequential(

          nn.Linear(100,100),
          nn.ReLU(inplace=True),
        nn.Dropout(p=0.5, inplace=False),
            nn.Linear(100,100),
        )

# lin_add = LinearAddLayer(100,100)

# model.add_module('linear_add',lin_add)
model.add_module('location_fc',location_fc)
# model.add_module('location_fc1',location_fc1)

# model.add_module('location_fc2',location_fc2)

def forward(self, x: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
    # Reshape and permute the input tensor
    x = self._forward_impl(x)
    # x = self.location_fc1(x)
    # x = self.linear_add(x,x5)
    
    x2 = self.location_fc1(x2)
    combined = torch.cat((x.view(x.size(0), -1),
                      x2.view(x2.size(0), -1)),
                      dim=1)
    x = self.location_fc(combined)
    return x

import functools


# model.forward = forward
model.forward = functools.partial(forward, model)




USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()
    
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')

# freezing trained part
for param in model.parameters():
    param.requires_grad = False

for param in model.location_fc.parameters():
    param.requires_grad = True
for param in model.location_fc1.parameters():
    param.requires_grad = True
      
    
if USE_CUDA:
    model.cuda()
# train_model_square(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 1)
train_model(model,LEARNING_RATE = 1e-6,NUM_EPOCHS = 10)
# train_model(model,LEARNING_RATE = 1e-6,NUM_EPOCHS = 20)

Using cache found in C:\Users\vjosv/.cache\torch\hub\pytorch_vision_v0.10.0


=> loading checkpoint 'C:/Users/vjosv/master/Deep-Leafsnap/saved_models/resnet101_v4_100targets_weights_seed1711_split8020_attempt2_model_best.pth.tar'
=> loaded checkpoint 'C:/Users/vjosv/master/Deep-Leafsnap/saved_models/resnet101_v4_100targets_weights_seed1711_split8020_attempt2_model_best.pth.tar' (epoch 17)
=> no checkpoint found at 'saved_models/resnet101_v4_100targets_weights_seed1711_split8020_attempt2_location3_checkpoint.pth.tar'

[INFO] Training Started

[Learning Rate] 0.000001
Epoch: [0][0/1334]	\Time 0.434 (0.434)	Data 0.255 (0.255)	Loss 38443.2383 (38443.2383)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [0][100/1334]	\Time 0.296 (0.310)	Data 0.224 (0.242)	Loss 4.8937 (1294.6233)	Prec@1 0.000 (1.073)	Prec@5 0.000 (4.868)
Epoch: [0][200/1334]	\Time 0.389 (0.320)	Data 0.316 (0.251)	Loss 4.5914 (652.8622)	Prec@1 0.000 (0.912)	Prec@5 8.333 (4.643)
Epoch: [0][300/1334]	\Time 0.599 (0.326)	Data 0.530 (0.257)	Loss 4.6345 (437.5163)	Prec@1 0.000 (0.969)	Prec@5 0.000 (4.734)


KeyboardInterrupt: 

In [110]:
model = torchvision.models.vit_b_16(weights = 'ViT_B_16_Weights.IMAGENET1K_V1')

seed =1711
split_train =80
split_val =100-split_train
numb_targets = 100

torch.cuda.empty_cache() 


optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()

# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

model.heads[0] =nn.Linear(768 , 100, bias = True)


model.name = f'vit_b_16_v6_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2'


if USE_CUDA:
    model.cuda()

# args_resume = f'C:/Users/vjosv/master/Deep-Leafsnap/saved_models/{model.name}_model_best.pth.tar'
# print("=> loading checkpoint '{}'".format(args_resume))
# checkpoint = torch.load(args_resume)
# best_prec1 = checkpoint['best_prec1']
# model.load_state_dict(checkpoint['state_dict'])
# print("=> loaded checkpoint '{}' (epoch {})"
#       .format(args_resume, checkpoint['epoch']))
# epoch = checkpoint['epoch']

test_path ='C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_test.csv'

train_path =  'C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_train.csv'
# test_dat_path = 'C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_landscape_kde_test.csv'

train_image_dataset = ImagesWithLocationDataset(train_path,'s',transform=composed)
test_image_dataset = ImagesWithLocationDataset(test_path,'s',transform=composed)

torch.manual_seed(seed)
# train_size = int(split_train*0.01 * len(image_dataset))
# test_size = len(image_dataset) - train_size
# data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()

@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS):

    train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=12, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=12, shuffle=False, num_workers=0)
    
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    print('\n[INFO] Training Started')
    
    for param_group in optimizer.param_groups:
        print(param_group['lr'])
    five_epochs_since_best = False
    epochs_since_best = 0
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
            
        adjusted_rate,LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best,LEARNING_RATE)
        print()
        if adjusted_rate:
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)
        # prec1 = top1.avg
        # prec5 = top5.avg
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        # torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1
            

            

if USE_CUDA:
    model.cuda()

args_resume = f'C:/Users/vjosv/master/Deep-Leafsnap/saved_models/{model.name}_model_best.pth.tar'
print("=> loading checkpoint '{}'".format(args_resume))
checkpoint = torch.load(args_resume)
best_prec1 = checkpoint['best_prec1']
model.load_state_dict(checkpoint['state_dict'])
print("=> loaded checkpoint '{}' (epoch {})"
      .format(args_resume, checkpoint['epoch']))
epoch = checkpoint['epoch']


model.name = f'vit_b_16_v6_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2_location'

            
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')

             
            
numb_targets = 100
location_vals = 170
location_fc = nn.Sequential(
          nn.Linear(numb_targets+location_vals, (numb_targets+location_vals)),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.5, inplace=False),
          nn.Linear((numb_targets+location_vals),(numb_targets+location_vals)),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.5, inplace=False),
            nn.Linear((numb_targets+location_vals),(numb_targets+location_vals)),
              nn.ReLU(inplace=True),
          nn.Dropout(p=0.5, inplace=False),
            nn.Linear((numb_targets+location_vals),numb_targets),
        )

model.add_module('location_fc',location_fc)




location_fc1 = nn.Sequential(
          nn.Linear(100, 100),
          nn.ReLU(inplace=True),
          nn.Dropout(p=0.5, inplace=False),
          nn.Linear(100,100),
        )

location_fc2 = nn.Sequential(

          nn.Linear(100,100),
          nn.ReLU(inplace=True),
        nn.Dropout(p=0.5, inplace=False),
            nn.Linear(100,100),
        )

# lin_add = LinearAddLayer(100,100)

# model.add_module('linear_add',lin_add)
model.add_module('location_fc',location_fc)
# model.add_module('location_fc1',location_fc1)

# model.add_module('location_fc2',location_fc2)

def forward(self, x: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
    # Reshape and permute the input tensor
    x = self._process_input(x)
    n = x.shape[0]

    # Expand the class token to the full batch
    batch_class_token = self.class_token.expand(n, -1, -1)
    x = torch.cat([batch_class_token, x], dim=1)

    x = self.encoder(x)

    # Classifier "token" as used by standard language architectures
    x = x[:, 0]

    x = self.heads(x)

    
    combined = torch.cat((x.view(x.size(0), -1),
                      x2.view(x2.size(0), -1)),
                      dim=1)

    x = self.location_fc(combined)
    
    return x
    

import functools


# model.forward = forward
model.forward = functools.partial(forward, model)




USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()
    
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')

#freezing transformer part
for param in model.parameters():
    param.requires_grad = False

for param in model.location_fc.parameters():
    param.requires_grad = True
    
    
if USE_CUDA:
    model.cuda()
# train_model_square(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 1)
train_model(model,LEARNING_RATE = 1e-4,NUM_EPOCHS = 1)
train_model(model,LEARNING_RATE = 1e-5,NUM_EPOCHS = 10)
train_model(model,LEARNING_RATE = 1e-6,NUM_EPOCHS = 20) 
train_model(model,LEARNING_RATE = 1e-7,NUM_EPOCHS = 30) 

=> loading checkpoint 'C:/Users/vjosv/master/Deep-Leafsnap/saved_models/vit_b_16_v6_100targets_weights_seed1711_split8020_attempt2_model_best.pth.tar'
=> loaded checkpoint 'C:/Users/vjosv/master/Deep-Leafsnap/saved_models/vit_b_16_v6_100targets_weights_seed1711_split8020_attempt2_model_best.pth.tar' (epoch 17)
=> loading checkpoint 'saved_models/vit_b_16_v6_100targets_weights_seed1711_split8020_attempt2_location_checkpoint.pth.tar'
=> loaded checkpoint 'saved_models/vit_b_16_v6_100targets_weights_seed1711_split8020_attempt2_location_checkpoint.pth.tar' (epoch 10)

[INFO] Training Started
1.0000000000000002e-07
=> loading checkpoint 'saved_models/vit_b_16_v6_100targets_weights_seed1711_split8020_attempt2_location_checkpoint.pth.tar'
=> loaded checkpoint 'saved_models/vit_b_16_v6_100targets_weights_seed1711_split8020_attempt2_location_checkpoint.pth.tar' (epoch 10)

[INFO] Training Started
1.0000000000000002e-07
=> loading checkpoint 'saved_models/vit_b_16_v6_100targets_weights_seed1711_

C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torch\nn\functional.py:5440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:235.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch: [10][0/1334]	\Time 0.493 (0.493)	Data 0.323 (0.323)	Loss 4.5686 (4.5686)	Prec@1 8.333 (8.333)	Prec@5 25.000 (25.000)
Epoch: [10][100/1334]	\Time 0.349 (0.412)	Data 0.202 (0.261)	Loss 4.6027 (4.6055)	Prec@1 0.000 (1.320)	Prec@5 0.000 (5.611)
Epoch: [10][200/1334]	\Time 0.462 (0.408)	Data 0.304 (0.259)	Loss 4.5971 (4.6054)	Prec@1 0.000 (1.285)	Prec@5 8.333 (5.307)
Epoch: [10][300/1334]	\Time 0.460 (0.415)	Data 0.305 (0.266)	Loss 4.5950 (4.6054)	Prec@1 8.333 (1.190)	Prec@5 16.667 (5.648)
Epoch: [10][400/1334]	\Time 0.490 (0.417)	Data 0.329 (0.269)	Loss 4.6283 (4.6056)	Prec@1 0.000 (1.205)	Prec@5 0.000 (5.424)
Epoch: [10][500/1334]	\Time 0.437 (0.419)	Data 0.287 (0.270)	Loss 4.5965 (4.6053)	Prec@1 0.000 (1.248)	Prec@5 0.000 (5.589)
Epoch: [10][600/1334]	\Time 0.458 (0.421)	Data 0.309 (0.272)	Loss 4.6119 (4.6056)	Prec@1 0.000 (1.137)	Prec@5 0.000 (5.491)
Epoch: [10][700/1334]	\Time 0.388 (0.429)	Data 0.288 (0.280)	Loss 4.6131 (4.6056)	Prec@1 0.000 (1.106)	Prec@5 0.000 (5.421)
Epoch: 

C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torch\nn\modules\activation.py:1196: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:235.)
  return torch._native_multi_head_attention(


Test: [0/334]

Time 0.318 (0.318)

Loss 4.6026 (4.6026)

Prec@1 0.000 (0.000)

Prec@5 8.333 (8.333)

Test: [100/334]

Time 0.412 (0.505)

Loss 4.6166 (4.6069)

Prec@1 0.000 (0.825)

Prec@5 0.000 (4.208)

Test: [200/334]

Time 0.156 (0.462)

Loss 4.6014 (4.6040)

Prec@1 0.000 (0.746)

Prec@5 0.000 (6.385)

Test: [300/334]

Time 0.144 (0.462)

Loss 4.6007 (4.6052)

Prec@1 0.000 (0.969)

Prec@5 0.000 (5.371)


[INFO] Saved Model to leafsnap_model.pth

[Learning Rate] 0.000001

Epoch: [11][0/1334]	\Time 0.960 (0.960)	Data 0.321 (0.321)	Loss 4.6127 (4.6127)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [11][100/1334]	\Time 0.482 (0.539)	Data 0.332 (0.268)	Loss 4.6101 (4.6071)	Prec@1 0.000 (0.825)	Prec@5 0.000 (4.950)
Epoch: [11][200/1334]	\Time 0.495 (0.550)	Data 0.338 (0.274)	Loss 4.5960 (4.6062)	Prec@1 0.000 (0.871)	Prec@5 8.333 (5.431)
Epoch: [11][300/1334]	\Time 0.404 (0.558)	Data 0.255 (0.273)	Loss 4.6148 (4.6062)	Prec@1 0.000 (0.941)	Prec@5 8.333 (5.122)
Epoch: [11][400/1334]	\Time

In [ ]:
# model.forward??25188
model.classifier[0] = nn.Linear(25188, 4096)

model.classifier[-1] = nn.Linear(4096, 100)

In [ ]:
def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        combined = torch.cat((x.view(x.size(0), -1),
                          y.view(y.size(0), -1)), dim=1)
        x = self.classifier(combined)
        return x
import functools


model.forward = functools.partial(forward, model)



In [32]:
train_loader = torch.utils.data.DataLoader(data_train, batch_size=16, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(data_val, batch_size=16, shuffle=False, num_workers=0)


In [ ]:
location_dataframe = pd.read_csv('C:/Users/vjosv/master/top_100_images_with_location_data.csv')
target = np.array(location_dataframe['map_square'].iloc[4])

target

In [ ]:
for i, j in enumerate(data_train):
    
    if i ==0:
        break

In [ ]:


j['image'].shape

In [ ]:
transforms = [T.ToTensor(),T.Resize((INPUT_SIZE,INPUT_SIZE))]

for t in transforms:

    if isinstance(t,torchvision.transforms.transforms.Resize):
        print('v')
        input_size = t.size[0]

s = torchvision.transforms.transforms.Resize((64,64))

s.size[]

In [ ]:
model.name='vgg19_artsobservasjoner64_with_location_data1'
# model.classifier.ins


USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
    

In [9]:
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred_t = pred.t()

    correct = pred_t.eq(target.view(1, -1).expand_as(pred_t))

    nb_classes = output.shape[1]

    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    _, preds = torch.max(output, 1)
    
    for t, p in zip(target.view(-1), preds.view(-1)):
        confusion_matrix[t.long(), p.long()] += 1

    # print(confusion_matrix)
    
    
    # print(confusion_matrix.diag()/confusion_matrix.sum(1))
        
    res = []
    for k in topk:
        correct_k = torch.sum(torch.reshape(correct[:k],(-1,) ).float()) #.double().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    res.append(confusion_matrix)
    return res

NameError: name 'model' is not defined

In [10]:
start_epoch = 0

args_resume = f'{model.name}_model_best.pth.tar'
# args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
# if args_resume:
#     if os.path.isfile(args_resume):
#         print("=> loading checkpoint '{}'".format(args_resume))
#         checkpoint = torch.load(args_resume)
#         start_epoch = checkpoint['epoch']
#         best_prec1 = checkpoint['best_prec1']
#         model.load_state_dict(checkpoint['state_dict'])
#         optimizer.load_state_dict(checkpoint['optimizer'])
#         print("=> loaded checkpoint '{}' (epoch {})"
#               .format(args_resume, checkpoint['epoch']))
#         start_epoch = checkpoint['epoch']
#     else:
#         print("=> no checkpoint found at '{}'".format(args_resume))

# print('\n[INFO] Training Started')
args_resume = None

start_epoch =0
@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS):
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
    train_loader = torch.utils.data.DataLoader(data_train, batch_size=16, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(data_val, batch_size=16, shuffle=False, num_workers=0)
    
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    print('\n[INFO] Training Started')
    five_epochs_since_best = False
    epochs_since_best = 0
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
            
        adjusted_rate,LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best,LEARNING_RATE)
        
        if adjusted_rate:
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1
        
# train_model(model)

NameError: name 'model' is not defined

In [11]:
model = torchvision.models.vgg19(pretrained=False)


In [12]:
model.name='vgg19_artsobservasjoner64_with_location_data2'
# model.classifier.ins


USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
    

In [15]:
#modify vgg19

# model.forward??25188
model.classifier[0] = nn.Linear(25188, 4096)

model.classifier[-1] = nn.Linear(4096, 100)

location_fc = nn.Sequential(
          nn.Linear(100, 100),
          nn.ReLU(inplace=True),
          nn.Linear(100,100),
          nn.ReLU(inplace=True),
            nn.Linear(100,100),
          nn.ReLU(inplace=True)
        )

model.add_module('location_fc',location_fc)


def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        y = self.location_fc(y)
        combined = torch.cat((x.view(x.size(0), -1),
                          y.view(y.size(0), -1)), dim=1)
        x = self.classifier(combined)
        return x
import functools


model.forward = functools.partial(forward, model)


AttributeError: 'VisionTransformer' object has no attribute 'classifier'

In [ ]:
if USE_CUDA:
    model.cuda()
train_model(model)

In [ ]:


model = torchvision.models.resnet101(weights=None)

model.name='resnet101_artsobservasjoner64_with_location_data'
# model.classifier.ins


USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
    
model.fc = nn.Linear(2048, 100)

In [ ]:
model.conv1 = nn.Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [ ]:

def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:

        x = torch.cat((x,y),1)
        
        return self._forward_impl(x)

import functools


model.forward = functools.partial(forward, model)

    
if USE_CUDA:
    model.cuda()

In [12]:
def train(train_loader, model, criterion, optimizer, epoch):

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, input in enumerate(train_loader):
        # measure data loading time
        # print(input['image'].shape)
        if USE_CUDA:
            image = input['image'].cuda(non_blocking=True)
            target = input['target'].cuda(non_blocking=True)
            # location_data = input['location_data'].cuda(non_blocking=True)
            # map_square = input['map_square'].cuda(non_blocking=True)
        else:
            image = input['image']
            target = input['target']
            # location_data = input['location_data']
            # map_square = input['map_square']
            
        input_var = torch.autograd.Variable(image)
        target_var = torch.autograd.Variable(target)
        # map_square_var = torch.autograd.Variable(map_square)
        # location_data_var = torch.autograd.Variable(location_data)
        # compute output
        # output = model(input_var,map_square)
        output = model(input_var)
        data_time.update(time.time() - end)


        # compute output
        # print(input_var.shape)
        # print(output)

        loss = criterion(output, target_var)
        # print(loss)
        # if i >10:
        #     break
        
        # save_output_target(model,output,target_var,epoch,i)
        # measure accuracy and record loss
        prec1, prec5,conf = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data.item(), image.size(0))
        top1.update(prec1.item(), image.size(0))
        top5.update(prec5.item(), image.size(0))
        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  '\Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1, top5=top5))
            
def validate(val_loader, model, criterion,epoch,save_output=False):
    # global output_aug_kde001, sample_pos, within_square, weight_tensor_kde001,zero_tensor_kde001,tt
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()


    conf = AverageMeter()
    class_correct = list(0. for i in range(185))
    class_total = list(0. for i in range(185))

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, input in enumerate(val_loader):
        

        if USE_CUDA:
            image = input['image'].cuda(non_blocking=True)
            target = input['target'].cuda(non_blocking=True)
            # location_data = input['location_data'].cuda(non_blocking=True)
            # map_square = input['map_square'].cuda(non_blocking=True)
        else:
            image = input['image']
            target = input['target']
            # location_data = input['location_data']
            # map_square = input['map_square']
        input_var = torch.autograd.Variable(image)
        target_var = torch.autograd.Variable(target)
        # location_data_var = torch.autograd.Variable(location_data)
        # map_square_var = torch.autograd.Variable(map_square)
        # compute output
        # output = model(input_var,map_square)
        output = model(input_var)
        loss = criterion(output, target)
        
        save_output_target_validate(model,output,target,epoch,i)
        

        # measure accuracy and record loss
        
        prec1, prec5, conf1 = accuracy(output.data, target, topk=(1, 5))
        

        losses.update(loss.data.item(), image.size(0))
        
        top1.update(prec1.item(), image.size(0))
        top5.update(prec5.item(), image.size(0))


        conf.update(conf1,1)
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 10 == 0:

            print('Test: [{0}/{1}]\n'.format(i, len(val_loader)))
            print('Time {batch_time.val:.3f} ({batch_time.avg:.3f})\n'.format(batch_time=batch_time))
            print('Loss {loss.val:.4f} ({loss.avg:.4f})\n'.format(loss=losses))
            print('Prec@1 {top1.val:.3f} ({top1.avg:.3f})\n'.format(top1=top1))
            print('Prec@5 {top5.val:.3f} ({top5.avg:.3f})\n'.format(top5=top5))

        print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
        


    return top1.avg


In [13]:
image_dataset = ImagesWithLocationDataset_conv('C:/Users/vjosv/master/top_100_images_with_location_data.csv','s',transform=composed)

train_size = int(0.9 * len(image_dataset))
test_size = len(image_dataset) - train_size
data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


start_epoch =0
@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS):
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
    train_loader = torch.utils.data.DataLoader(data_train, batch_size=16, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(data_val, batch_size=16, shuffle=False, num_workers=0)
    
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    print('\n[INFO] Training Started')
    five_epochs_since_best = False
    epochs_since_best = 0
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
            
        adjusted_rate,LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best, LEARNING_RATE)
        
        if adjusted_rate:
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1
        

In [ ]:
if USE_CUDA:
    model.cuda()
train_model(model)

In [16]:
model = torchvision.models.vit_b_16(weights = None)


In [17]:
model.heads[0] = nn.Linear(768 , 150, bias = True)
model.name = 'vit_b_16_artsobservasjoner224_150targets'

if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
    

In [13]:
image_dataset = ImagesWithLocationDataset('C:/Users/vjosv/master/top_150_images_with_location_data.csv','s',transform=composed)

torch.manual_seed(2610)
train_size = int(0.9 * len(image_dataset))
test_size = len(image_dataset) - train_size
data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])

C:\Users\vjosv\AppData\Local\Temp\ipykernel_22400\3414169806.py:13: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  self.location_dataframe = pd.read_csv(csv_file)


In [33]:
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred_t = pred.t()

    correct = pred_t.eq(target.view(1, -1).expand_as(pred_t))

    nb_classes = output.shape[1]

    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    _, preds = torch.max(output, 1)
    
    for t, p in zip(target.view(-1), preds.view(-1)):
        confusion_matrix[t.long(), p.long()] += 1

    # print(confusion_matrix)
    
    
    # print(confusion_matrix.diag()/confusion_matrix.sum(1))
        
    res = []
    for k in topk:
        correct_k = torch.sum(torch.reshape(correct[:k],(-1,) ).float()) #.double().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    res.append(confusion_matrix)
    return res

In [12]:
import copy

def train(train_loader, model, criterion, optimizer, epoch):

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, input in enumerate(train_loader):
        # measure data loading time
        # print(input['image'].shape)
        if USE_CUDA:
            image = input['image'].cuda(non_blocking=True)
            target = input['target'].cuda(non_blocking=True)
            # location_data = input['location_data'].cuda(non_blocking=True)
            # map_square = input['map_square'].cuda(non_blocking=True)
        else:
            image = input['image']
            target = input['target']
            # location_data = input['location_data']
            # map_square = input['map_square']
            
        input_var = torch.autograd.Variable(image)
        target_var = torch.autograd.Variable(target)
        # map_square_var = torch.autograd.Variable(map_square)
        # location_data_var = torch.autograd.Variable(location_data)
        # compute output
        # output = model(input_var,map_square)
        output = model(input_var)
        data_time.update(time.time() - end)


        # compute output
        # print(input_var.shape)
        # print(output)

        loss = criterion(output, target_var)
        # print(loss)
        # if i >10:
        #     break
        
        # save_output_target(model,output,target_var,epoch,i)
        # measure accuracy and record loss
        prec1, prec5,conf = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data.item(), image.size(0))
        top1.update(prec1.item(), image.size(0))
        top5.update(prec5.item(), image.size(0))
        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  '\Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1, top5=top5))
            
def validate(val_loader, model, criterion,epoch,save_output=False):
    # global output_aug_kde001, sample_pos, within_square, weight_tensor_kde001,zero_tensor_kde001,tt
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()


    conf = AverageMeter()
    class_correct = list(0. for i in range(185))
    class_total = list(0. for i in range(185))

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, input in enumerate(val_loader):
        

        if USE_CUDA:
            image = input['image'].cuda(non_blocking=True)
            target = input['target'].cuda(non_blocking=True)
            # location_data = input['location_data'].cuda(non_blocking=True)
            # map_square = input['map_square'].cuda(non_blocking=True)
        else:
            image = input['image']
            target = input['target']
            # location_data = input['location_data']
            # map_square = input['map_square']
        input_var = torch.autograd.Variable(image)
        target_var = torch.autograd.Variable(target)
        # location_data_var = torch.autograd.Variable(location_data)
        # map_square_var = torch.autograd.Variable(map_square)
        # compute output
        # output = model(input_var,map_square)
        output = model(input_var)
        loss = criterion(output, target)
        
        save_output_target_validate(model,output,target,epoch,i)
        

        # measure accuracy and record loss
        
        prec1, prec5, conf1 = accuracy(output.data, target, topk=(1, 5))
        

        losses.update(loss.data.item(), image.size(0))
        
        top1.update(prec1.item(), image.size(0))
        top5.update(prec5.item(), image.size(0))


        conf.update(conf1,1)
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 10 == 0:

            print('Test: [{0}/{1}]\n'.format(i, len(val_loader)))
            print('Time {batch_time.val:.3f} ({batch_time.avg:.3f})\n'.format(batch_time=batch_time))
            print('Loss {loss.val:.4f} ({loss.avg:.4f})\n'.format(loss=losses))
            print('Prec@1 {top1.val:.3f} ({top1.avg:.3f})\n'.format(top1=top1))
            print('Prec@5 {top5.val:.3f} ({top5.avg:.3f})\n'.format(top5=top5))

        print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
        


    return top1.avg

def adjust_learning_rate(optimizer, epoch, adjust_now, LEARNING_RATE=LEARNING_RATE):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LEARNING_RATE
    if (epoch-1) < 0:
        former_lr=LEARNING_RATE
    # else:
        # former_lr = LEARNING_RATE * (0.1 ** ((epoch-1) // 10))
    # lr = LEARNING_RATE * (0.1 ** (epoch // 10))
    former_lr = copy.copy(LEARNING_RATE)
    if adjust_now:
        former_lr = copy.copy(LEARNING_RATE)
        LEARNING_RATE = LEARNING_RATE*0.1
        lr = LEARNING_RATE
    if (lr <= 0.0001):
        lr = 0.0001
    print('\n[Learning Rate] {:0.6f}'.format(lr))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr != former_lr, LEARNING_RATE


def save_checkpoint(state, is_best, model):
    filename=f'saved_models/{model.name}_checkpoint.pth.tar'
    torch.save(state, filename)
    if is_best:
        print('\n[INFO] Saved Model to model_best.pth.tar')
        shutil.copyfile(filename, f'saved_models/{model.name}_model_best.pth.tar')
        
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred_t = pred.t()

    correct = pred_t.eq(target.view(1, -1).expand_as(pred_t))

    nb_classes = output.shape[1]

    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    _, preds = torch.max(output, 1)
    
    for t, p in zip(target.view(-1), preds.view(-1)):
        confusion_matrix[t.long(), p.long()] += 1

    # print(confusion_matrix)
    
    
    # print(confusion_matrix.diag()/confusion_matrix.sum(1))
        
    res = []
    for k in topk:
        correct_k = torch.sum(torch.reshape(correct[:k],(-1,) ).float()) #.double().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    res.append(confusion_matrix)
    return res

In [13]:
start_epoch =0
@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS,init_learning_rate = None,load_best=False ):
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
    train_loader = torch.utils.data.DataLoader(data_train, batch_size=16, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(data_val, batch_size=16, shuffle=False, num_workers=0)
    
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    print('\n[INFO] Training Started')
    five_epochs_since_best = False
    epochs_since_best = 0
    first_run = True 
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
        
        if first_run and init_learning_rate:
            LEARNING_RATE = init_learning_rate
            print(LEARNING_RATE)
            adjusted_rate= False
        else:
            adjusted_rate, LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best,LEARNING_RATE=LEARNING_RATE)
            five_epochs_since_best = False
        if adjusted_rate or (load_best and first_run):
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1
            
        first_run = False

def save_output_target_validate(model,output,target,epoch,batch):
        torch.save(output,f'saved_output/{model.name}_validating_output/output_e{epoch}_b{batch}')
        torch.save(target,f'saved_output/{model.name}_validating_output/target_b{batch}')
   

In [24]:
if USE_CUDA:
    model.cuda()
train_model(model)

=> loading checkpoint 'saved_models/vit_b_16_artsobservasjoner224_100targets_weights_Noneseed2_checkpoint.pth.tar'
=> loaded checkpoint 'saved_models/vit_b_16_artsobservasjoner224_100targets_weights_Noneseed2_checkpoint.pth.tar' (epoch 24)

[INFO] Training Started

[Learning Rate] 0.100000


KeyboardInterrupt: 

In [ ]:
model = torchvision.models.vit_b_16(weights = True)
model.heads[0] = nn.Linear(768 , 150, bias = True)
model.name = 'vit_b_16_artsobservasjoner224_150targets_weights_None'

if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 20)

In [ ]:
model = torchvision.models.vit_b_16(weights = True)
model.heads[0] = nn.Linear(768 , 100, bias = True)
model.name = 'vit_b_16_artsobservasjoner224_100targets_pretrained'

image_dataset = ImagesWithLocationDataset('C:/Users/vjosv/master/top_100_images_with_location_data.csv','s',transform=composed)

train_size = int(0.9 * len(image_dataset))
test_size = len(image_dataset) - train_size
data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 20)

In [ ]:
model = torchvision.models.vit_b_16(weights = True)
model.heads[0] = nn.Linear(768 , 125, bias = True)
model.name = 'vit_b_16_artsobservasjoner224_125targets_pretrained'

image_dataset = ImagesWithLocationDataset('C:/Users/vjosv/master/top_125_images_with_location_data.csv','s',transform=composed)

train_size = int(0.9 * len(image_dataset))
test_size = len(image_dataset) - train_size
data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 20)

In [13]:
model = torchvision.models.vit_b_16(weights = True)


model.heads[0] = nn.Linear(768 , 100, bias = True)
model.name = 'vit_b_16_artsobservasjoner224_100targets_weights_seed0802split'

image_dataset = ImagesWithLocationDataset('C:/Users/vjosv/master/top_100_images_with_location_data.csv','s',transform=composed)

torch.manual_seed(2610)
train_size = int(0.8 * len(image_dataset))
test_size = len(image_dataset) - train_size
data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 30)

C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


=> no checkpoint found at 'saved_models/vit_b_16_artsobservasjoner224_100targets_weights_seed0802split_checkpoint.pth.tar'

[INFO] Training Started

[Learning Rate] 0.010000


C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torch\nn\functional.py:5440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:235.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch: [0][0/558]	\Time 0.795 (0.795)	Data 0.421 (0.421)	Loss 4.8819 (4.8819)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [0][100/558]	\Time 0.499 (0.500)	Data 0.410 (0.399)	Loss 4.6533 (4.6105)	Prec@1 0.000 (2.413)	Prec@5 6.250 (10.891)
Epoch: [0][200/558]	\Time 0.574 (0.512)	Data 0.464 (0.411)	Loss 4.7186 (4.5698)	Prec@1 0.000 (2.736)	Prec@5 0.000 (12.002)
Epoch: [0][300/558]	\Time 0.532 (0.524)	Data 0.422 (0.422)	Loss 4.4500 (4.5452)	Prec@1 0.000 (2.845)	Prec@5 6.250 (12.334)
Epoch: [0][400/558]	\Time 0.581 (0.530)	Data 0.471 (0.427)	Loss 4.2562 (4.5192)	Prec@1 6.250 (2.993)	Prec@5 25.000 (12.703)
Epoch: [0][500/558]	\Time 0.559 (0.533)	Data 0.449 (0.431)	Loss 4.7800 (4.4873)	Prec@1 0.000 (3.293)	Prec@5 6.250 (13.373)
Test: [0/140]

Time 0.530 (0.530)

Loss 4.7040 (4.7040)

Prec@1 0.000 (0.000)

Prec@5 6.250 (6.250)

 * Prec@1 0.000 Prec@5 6.250
 * Prec@1 3.125 Prec@5 15.625
 * Prec@1 2.083 Prec@5 22.917
 * Prec@1 1.562 Prec@5 20.312
 * Prec@1 1.250 Prec@5 20.000
 * Prec@1 1.04

In [18]:
model = torchvision.models.vit_b_16(weights = None)


model.heads[0] = nn.Linear(768 , 125, bias = True)
model.name = 'vit_b_16_artsobservasjoner224_125targets_weights_Noneseed'

image_dataset = ImagesWithLocationDataset('C:/Users/vjosv/master/top_125_images_with_location_data.csv','s',transform=composed)

torch.manual_seed(2610)
train_size = int(0.9 * len(image_dataset))
test_size = len(image_dataset) - train_size
data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 20)

C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


=> no checkpoint found at 'saved_models/vit_b_16_artsobservasjoner224_125targets_weights_Noneseed_checkpoint.pth.tar'

[INFO] Training Started

[Learning Rate] 0.010000


C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torch\nn\functional.py:5440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:235.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch: [0][0/728]	\Time 0.734 (0.734)	Data 0.449 (0.449)	Loss 4.8297 (4.8297)	Prec@1 0.000 (0.000)	Prec@5 6.250 (6.250)


KeyboardInterrupt: 

In [23]:
model = torchvision.models.vit_b_16(weights = True)

model.heads[0] = nn.Linear(768 , 200, bias = True)
model.name = 'vit_b_16_artsobservasjoner224_200targets_weights_seed2'

image_dataset = ImagesWithLocationDataset('C:/Users/vjosv/master/top_200_images_with_location_data.csv','s',transform=composed)


torch.manual_seed(2610)
train_size = int(0.9 * len(image_dataset))
test_size = len(image_dataset) - train_size
data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 20)

C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\vjosv\AppData\Local\Temp\ipykernel_19992\3414169806.py:13: DtypeWarning: Columns (38,54,60) have mixed types. Specify dtype option on import or set low_memory=False.
  self.location_dataframe = pd.read_csv(csv_file)


=> loading checkpoint 'saved_models/vit_b_16_artsobservasjoner224_200targets_weights_seed2_checkpoint.pth.tar'
=> loaded checkpoint 'saved_models/vit_b_16_artsobservasjoner224_200targets_weights_seed2_checkpoint.pth.tar' (epoch 1)

[INFO] Training Started

[Learning Rate] 0.010000


C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torch\nn\functional.py:5440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:235.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch: [1][0/987]	\Time 0.743 (0.743)	Data 0.381 (0.381)	Loss 5.0149 (5.0149)	Prec@1 12.500 (12.500)	Prec@5 18.750 (18.750)
Epoch: [1][100/987]	\Time 0.505 (0.490)	Data 0.388 (0.390)	Loss 4.8673 (4.5374)	Prec@1 0.000 (7.797)	Prec@5 6.250 (23.205)
Epoch: [1][200/987]	\Time 0.501 (0.495)	Data 0.380 (0.394)	Loss 4.7782 (4.4105)	Prec@1 0.000 (8.364)	Prec@5 12.500 (25.746)
Epoch: [1][300/987]	\Time 0.554 (0.512)	Data 0.438 (0.410)	Loss 4.0635 (4.3413)	Prec@1 12.500 (9.012)	Prec@5 31.250 (26.827)
Epoch: [1][400/987]	\Time 0.555 (0.519)	Data 0.444 (0.417)	Loss 4.4421 (4.2576)	Prec@1 18.750 (10.022)	Prec@5 25.000 (28.803)
Epoch: [1][500/987]	\Time 0.550 (0.525)	Data 0.440 (0.421)	Loss 3.2465 (4.2055)	Prec@1 18.750 (10.816)	Prec@5 62.500 (29.965)
Epoch: [1][600/987]	\Time 0.584 (0.527)	Data 0.481 (0.424)	Loss 4.4623 (4.1432)	Prec@1 12.500 (11.481)	Prec@5 25.000 (31.510)
Epoch: [1][700/987]	\Time 0.601 (0.531)	Data 0.491 (0.428)	Loss 3.6236 (4.0720)	Prec@1 12.500 (12.455)	Prec@5 31.250 (33.042)


In [18]:
model = torchvision.models.vit_b_16(weights = True)



model.heads[0] = nn.Linear(768 , 300, bias = True)
model.name = 'vit_b_16_artsobservasjoner224_300targets_weights_seed'

image_dataset = ImagesWithLocationDataset('C:/Users/vjosv/master/top_300_images_with_location_data.csv','s',transform=composed)

torch.manual_seed(2610)
train_size = int(0.9 * len(image_dataset))
test_size = len(image_dataset) - train_size
data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')

if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 20)

C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\vjosv\AppData\Local\Temp\ipykernel_19992\3414169806.py:13: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  self.location_dataframe = pd.read_csv(csv_file)


=> loading checkpoint 'saved_models/vit_b_16_artsobservasjoner224_300targets_weights_seed_checkpoint.pth.tar'
=> loaded checkpoint 'saved_models/vit_b_16_artsobservasjoner224_300targets_weights_seed_checkpoint.pth.tar' (epoch 17)

[INFO] Training Started

[Learning Rate] 0.010000


C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torch\nn\functional.py:5440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:235.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch: [17][0/1276]	\Time 0.838 (0.838)	Data 0.468 (0.468)	Loss 0.0023 (0.0023)	Prec@1 100.000 (100.000)	Prec@5 100.000 (100.000)
Epoch: [17][100/1276]	\Time 0.570 (0.536)	Data 0.460 (0.430)	Loss 0.0021 (0.0078)	Prec@1 100.000 (99.691)	Prec@5 100.000 (100.000)
Epoch: [17][200/1276]	\Time 0.550 (0.534)	Data 0.440 (0.430)	Loss 0.0027 (0.0085)	Prec@1 100.000 (99.720)	Prec@5 100.000 (100.000)
Epoch: [17][300/1276]	\Time 0.540 (0.536)	Data 0.430 (0.433)	Loss 0.0025 (0.0088)	Prec@1 100.000 (99.709)	Prec@5 100.000 (100.000)
Epoch: [17][400/1276]	\Time 0.598 (0.543)	Data 0.470 (0.440)	Loss 0.0013 (0.0094)	Prec@1 100.000 (99.688)	Prec@5 100.000 (100.000)
Epoch: [17][500/1276]	\Time 0.597 (0.553)	Data 0.486 (0.449)	Loss 0.0033 (0.0096)	Prec@1 100.000 (99.688)	Prec@5 100.000 (100.000)
Epoch: [17][600/1276]	\Time 0.541 (0.553)	Data 0.431 (0.450)	Loss 0.0027 (0.0091)	Prec@1 100.000 (99.709)	Prec@5 100.000 (100.000)
Epoch: [17][700/1276]	\Time 0.579 (0.554)	Data 0.479 (0.450)	Loss 0.0716 (0.0102)	Pr

In [17]:
model = torchvision.models.vit_b_16(weights = True)




model.heads[0] = nn.Linear(768 , 400, bias = True)
model.name = 'vit_b_16_artsobservasjoner224_400targets_weights_Noneseed'

image_dataset = ImagesWithLocationDataset('C:/Users/vjosv/master/top_400_images_with_location_data.csv','s',transform=composed)

torch.manual_seed(2610)
train_size = int(0.9 * len(image_dataset))
test_size = len(image_dataset) - train_size
data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 20)

C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\vjosv\AppData\Local\Temp\ipykernel_19992\3414169806.py:13: DtypeWarning: Columns (38,54) have mixed types. Specify dtype option on import or set low_memory=False.
  self.location_dataframe = pd.read_csv(csv_file)


=> loading checkpoint 'saved_models/vit_b_16_artsobservasjoner224_400targets_weights_Noneseed_checkpoint.pth.tar'
=> loaded checkpoint 'saved_models/vit_b_16_artsobservasjoner224_400targets_weights_Noneseed_checkpoint.pth.tar' (epoch 1)

[INFO] Training Started

[Learning Rate] 0.010000


C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torch\nn\functional.py:5440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:235.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch: [1][0/1515]	\Time 4.584 (4.584)	Data 3.899 (3.899)	Loss 2.9420 (2.9420)	Prec@1 31.250 (31.250)	Prec@5 62.500 (62.500)
Epoch: [1][100/1515]	\Time 0.509 (0.556)	Data 0.403 (0.449)	Loss 2.7180 (3.0243)	Prec@1 50.000 (34.777)	Prec@5 62.500 (60.705)
Epoch: [1][200/1515]	\Time 0.532 (0.553)	Data 0.424 (0.447)	Loss 2.7477 (2.9791)	Prec@1 31.250 (35.292)	Prec@5 62.500 (61.070)
Epoch: [1][300/1515]	\Time 0.580 (0.551)	Data 0.460 (0.447)	Loss 1.9841 (2.9450)	Prec@1 56.250 (35.424)	Prec@5 75.000 (62.022)
Epoch: [1][400/1515]	\Time 0.592 (0.558)	Data 0.481 (0.453)	Loss 2.8659 (2.8995)	Prec@1 25.000 (36.066)	Prec@5 62.500 (62.749)
Epoch: [1][500/1515]	\Time 0.579 (0.557)	Data 0.469 (0.452)	Loss 2.2432 (2.8484)	Prec@1 43.750 (37.188)	Prec@5 81.250 (63.348)
Epoch: [1][600/1515]	\Time 0.547 (0.556)	Data 0.447 (0.452)	Loss 1.8772 (2.8152)	Prec@1 43.750 (37.750)	Prec@5 75.000 (63.738)
Epoch: [1][700/1515]	\Time 0.532 (0.557)	Data 0.421 (0.453)	Loss 1.7446 (2.7411)	Prec@1 62.500 (39.149)	Prec@5 87

In [21]:
model = torchvision.models.vit_b_16(weights = None)



model.heads[0] = nn.Linear(768 , 300, bias = True)
model.name = 'vit_b_16_artsobservasjoner224_300targets_weights_none_seed'

image_dataset = ImagesWithLocationDataset('C:/Users/vjosv/master/top_300_images_with_location_data.csv','s',transform=composed)

torch.manual_seed(2610)
train_size = int(0.9 * len(image_dataset))
test_size = len(image_dataset) - train_size
data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')

if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 20)

C:\Users\vjosv\AppData\Local\Temp\ipykernel_19992\3414169806.py:13: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  self.location_dataframe = pd.read_csv(csv_file)


=> no checkpoint found at 'saved_models/vit_b_16_artsobservasjoner224_300targets_weights_none_seed_checkpoint.pth.tar'

[INFO] Training Started

[Learning Rate] 0.010000


C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torch\nn\functional.py:5440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:235.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch: [0][0/1276]	\Time 0.695 (0.695)	Data 0.472 (0.472)	Loss 5.9771 (5.9771)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [0][100/1276]	\Time 0.522 (0.521)	Data 0.409 (0.420)	Loss 6.2706 (6.5476)	Prec@1 0.000 (0.371)	Prec@5 0.000 (1.918)
Epoch: [0][200/1276]	\Time 0.512 (0.514)	Data 0.430 (0.415)	Loss 6.0646 (6.4451)	Prec@1 0.000 (0.498)	Prec@5 0.000 (2.799)
Epoch: [0][300/1276]	\Time 0.520 (0.513)	Data 0.400 (0.414)	Loss 5.7486 (6.2182)	Prec@1 0.000 (0.706)	Prec@5 0.000 (3.135)
Epoch: [0][400/1276]	\Time 0.575 (0.513)	Data 0.440 (0.413)	Loss 5.5703 (6.0892)	Prec@1 0.000 (0.810)	Prec@5 12.500 (3.382)
Epoch: [0][500/1276]	\Time 0.550 (0.515)	Data 0.440 (0.415)	Loss 5.9576 (6.0054)	Prec@1 0.000 (0.823)	Prec@5 0.000 (3.468)
Epoch: [0][600/1276]	\Time 0.531 (0.515)	Data 0.412 (0.415)	Loss 5.9811 (5.9495)	Prec@1 0.000 (0.915)	Prec@5 0.000 (3.785)
Epoch: [0][700/1276]	\Time 0.551 (0.514)	Data 0.441 (0.414)	Loss 5.6349 (5.9081)	Prec@1 0.000 (0.981)	Prec@5 0.000 (4.048)
Epoch: [0][800/12

In [15]:


'combined_location_image_df_top100.csv'


model = torchvision.models.vit_b_16(weights = True)




model.heads[0] = nn.Linear(768 , 100, bias = True)
model.name = 'vit_b_16_artsobservasjoner224_100targets_weights_seed211_split8020'

image_dataset = ImagesWithLocationDataset('combined_location_image_df_top100.csv','s',transform=composed)

torch.manual_seed(211)
train_size = int(0.8 * len(image_dataset))
test_size = len(image_dataset) - train_size
data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 20)

C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


=> no checkpoint found at 'saved_models/vit_b_16_artsobservasjoner224_100targets_weights_seed211_split8020_checkpoint.pth.tar'

[INFO] Training Started

[Learning Rate] 0.010000


C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torch\nn\functional.py:5440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:235.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch: [0][0/559]	\Time 1.804 (1.804)	Data 0.446 (0.446)	Loss 4.6828 (4.6828)	Prec@1 0.000 (0.000)	Prec@5 6.250 (6.250)
Epoch: [0][100/559]	\Time 0.476 (0.664)	Data 0.400 (0.516)	Loss 4.4492 (4.5853)	Prec@1 6.250 (3.589)	Prec@5 18.750 (10.953)
Epoch: [0][200/559]	\Time 0.554 (0.598)	Data 0.471 (0.477)	Loss 4.7595 (4.5624)	Prec@1 0.000 (3.296)	Prec@5 0.000 (11.816)
Epoch: [0][300/559]	\Time 0.535 (0.583)	Data 0.435 (0.469)	Loss 4.5231 (4.5510)	Prec@1 0.000 (3.322)	Prec@5 0.000 (11.898)
Epoch: [0][400/559]	\Time 0.491 (0.576)	Data 0.396 (0.465)	Loss 4.4330 (4.5251)	Prec@1 6.250 (3.538)	Prec@5 6.250 (12.547)
Epoch: [0][500/559]	\Time 0.480 (0.568)	Data 0.380 (0.460)	Loss 4.4567 (4.5030)	Prec@1 6.250 (3.505)	Prec@5 6.250 (13.111)
Test: [0/140]

Time 0.540 (0.540)

Loss 4.2261 (4.2261)

Prec@1 0.000 (0.000)

Prec@5 18.750 (18.750)

 * Prec@1 0.000 Prec@5 18.750
 * Prec@1 0.000 Prec@5 18.750
 * Prec@1 0.000 Prec@5 12.500
 * Prec@1 3.125 Prec@5 12.500
 * Prec@1 2.500 Prec@5 11.250
 * Prec@1 2

In [21]:
comb = pd.read_csv('top_200_images_with_location_data.csv')
import cv2
import numpy as np
import matplotlib.pyplot as plt



non_existng_files = []

for i in comb['path']:
    # print(i)
    if not os.path.isfile(i):
        non_existng_files.append(i)
    # Using cv2.imshow() method 
    # Displaying the image 

    # except:
        
non_existng_files

C:\Users\vjosv\AppData\Local\Temp\ipykernel_6172\1204856281.py:1: DtypeWarning: Columns (40,56,62) have mixed types. Specify dtype option on import or set low_memory=False.
  comb = pd.read_csv('top_200_images_with_location_data.csv')


[]

In [20]:

'combined_location_image_df_top100.csv'


model = torchvision.models.vit_b_16(weights = True)




model.heads[0] = nn.Linear(768 , 100, bias = True)
model.name = 'vit_b_16_artsobservasjoner224_200targets_weights_seed311_split8020'

image_dataset = ImagesWithLocationDataset('top_200_images_with_location_data.csv','s',transform=composed)

torch.manual_seed(311)
train_size = int(0.8 * len(image_dataset))
test_size = len(image_dataset) - train_size
data_train, data_val = torch.utils.data.random_split(image_dataset, [train_size, test_size])


if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 20)


C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\vjosv\AppData\Local\Temp\ipykernel_6172\3414169806.py:13: DtypeWarning: Columns (40,56,62) have mixed types. Specify dtype option on import or set low_memory=False.
  self.location_dataframe = pd.read_csv(csv_file)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
